In [1]:
from pyomo.environ import (
    Constraint,
    Var,
    ConcreteModel,
    Expression,
    Objective,
    TransformationFactory,
    value,
)
# Todo: Import the above mentioned tools from pyomo.network
from pyomo.network import Arc, SequentialDecomposition
from idaes.core import FlowsheetBlock

from idaes.models.unit_models import (
    PressureChanger,
    Mixer,
    Separator as Splitter,
    Heater,
    CSTR,
    Flash,
    Translator,
)

from idaes.models_extra.column_models import TrayColumn
from idaes.models_extra.column_models.condenser import CondenserType, TemperatureSpec
# Utility tools to put together the flowsheet and calculate the degrees of freedom
from idaes.models.unit_models.pressure_changer import ThermodynamicAssumption
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.initialization import propagate_state
from idaes.core.solvers import get_solver
import idaes.core.util.scaling as iscale

# Import idaes logger to set output levels
import idaes.logger as idaeslog

from idaes_examples.mod.hda import hda_reaction as reaction_props
from idaes.models.properties.activity_coeff_models.BTX_activity_coeff_VLE import (
    BTXParameterBlock,
)

from idaes_examples.mod.hda.hda_ideal_VLE import HDAParameterBlock
# Create a Pyomo Concrete Model to contain the problem
m = ConcreteModel()

# Add a steady state flowsheet block to the model
m.fs = FlowsheetBlock(dynamic=False)
# Property package for benzene, toluene, hydrogen, methane mixture
m.fs.BTHM_params = HDAParameterBlock()

# Property package for the benzene-toluene mixture
m.fs.BT_params = BTXParameterBlock(
    valid_phase=("Liq", "Vap"), activity_coeff_model="Ideal"
)

# Reaction package for the HDA reaction
m.fs.reaction_params = reaction_props.HDAReactionParameterBlock(
    property_package=m.fs.BTHM_params
)
# Adding the mixer M101 to the flowsheet
m.fs.M101 = Mixer(
    property_package=m.fs.BTHM_params,
    inlet_list=["toluene_feed", "hydrogen_feed", "vapor_recycle"],
)

# Adding the heater H101 to the flowsheet
m.fs.H101 = Heater(property_package=m.fs.BTHM_params, has_phase_equilibrium=True)
# Todo: Add reactor with the specifications above
m.fs.R101 = CSTR(
    property_package=m.fs.BTHM_params,
    reaction_package=m.fs.reaction_params,
    has_heat_of_reaction=True,
    has_heat_transfer=True,
)

# Adding the flash tank F101 to the flowsheet
m.fs.F101 = Flash(
    property_package=m.fs.BTHM_params, has_heat_transfer=True, has_pressure_change=True
)

# Adding the splitter S101 to the flowsheet
m.fs.S101 = Splitter(
    property_package=m.fs.BTHM_params, outlet_list=["purge", "recycle"]
)

# Adding the compressor C101 to the flowsheet
m.fs.C101 = PressureChanger(
    property_package=m.fs.BTHM_params,
    compressor=True,
    thermodynamic_assumption=ThermodynamicAssumption.isothermal,
)
# Add translator block to convert between property packages
m.fs.translator = Translator(
    inlet_property_package=m.fs.BTHM_params, outlet_property_package=m.fs.BT_params
)
# Add constraint: Total flow = benzene flow + toluene flow (molar)
m.fs.translator.eq_total_flow = Constraint(
    expr=m.fs.translator.outlet.flow_mol[0]
    == m.fs.translator.inlet.flow_mol_phase_comp[0, "Liq", "benzene"]
    + m.fs.translator.inlet.flow_mol_phase_comp[0, "Liq", "toluene"]
)

# Add constraint: Outlet temperature = Inlet temperature
m.fs.translator.eq_temperature = Constraint(
    expr=m.fs.translator.outlet.temperature[0] == m.fs.translator.inlet.temperature[0]
)

# Todo: Add constraint: Outlet pressure = Inlet pressure
m.fs.translator.eq_pressure = Constraint(
    expr=m.fs.translator.outlet.pressure[0] == m.fs.translator.inlet.pressure[0]
)

# Remaining constraints on the translator block

# Add constraint: Benzene mole fraction definition
m.fs.translator.eq_mole_frac_benzene = Constraint(
    expr=m.fs.translator.outlet.mole_frac_comp[0, "benzene"]
    == m.fs.translator.inlet.flow_mol_phase_comp[0, "Liq", "benzene"]
    / (
        m.fs.translator.inlet.flow_mol_phase_comp[0, "Liq", "benzene"]
        + m.fs.translator.inlet.flow_mol_phase_comp[0, "Liq", "toluene"]
        + m.fs.translator.inlet.flow_mol_phase_comp[0, "Liq", "hydrogen"]
        + m.fs.translator.inlet.flow_mol_phase_comp[0, "Liq", "methane"]
    )
)

# Add constraint: Toluene mole fraction definition
m.fs.translator.eq_mole_frac_toluene = Constraint(
    expr=m.fs.translator.outlet.mole_frac_comp[0, "toluene"]
    == m.fs.translator.inlet.flow_mol_phase_comp[0, "Liq", "toluene"]
    / (
        m.fs.translator.inlet.flow_mol_phase_comp[0, "Liq", "benzene"]
        + m.fs.translator.inlet.flow_mol_phase_comp[0, "Liq", "toluene"]
                + m.fs.translator.inlet.flow_mol_phase_comp[0, "Liq", "hydrogen"]
        + m.fs.translator.inlet.flow_mol_phase_comp[0, "Liq", "methane"]
    )
)
# Todo: Add the Heater H102 to the flowsheet
m.fs.H102 = Heater(
    property_package=m.fs.BT_params,
    has_pressure_change=True,
    has_phase_equilibrium=True,
)

m.fs.s03 = Arc(source=m.fs.M101.outlet, destination=m.fs.H101.inlet)

# Todo: Connect the H101 outlet to R101 inlet
m.fs.s04 = Arc(source=m.fs.H101.outlet, destination=m.fs.R101.inlet)
m.fs.s05 = Arc(source=m.fs.R101.outlet, destination=m.fs.F101.inlet)
m.fs.s06 = Arc(source=m.fs.F101.vap_outlet, destination=m.fs.S101.inlet)
m.fs.s08 = Arc(source=m.fs.S101.recycle, destination=m.fs.C101.inlet)
m.fs.s09 = Arc(source=m.fs.C101.outlet, destination=m.fs.M101.vapor_recycle)
m.fs.s10a = Arc(source=m.fs.F101.liq_outlet, destination=m.fs.translator.inlet)
m.fs.s10b = Arc(source=m.fs.translator.outlet, destination=m.fs.H102.inlet)

TransformationFactory("network.expand_arcs").apply_to(m)

# Define the conversion variables using 'Var'
m.fs.R101.conversion = Var(initialize=0.75, bounds=(0, 1))

# Append the constraint to the model
m.fs.R101.conv_constraint = Constraint(
    expr=m.fs.R101.conversion * m.fs.R101.inlet.flow_mol_phase_comp[0, "Vap", "toluene"]
    == (
        m.fs.R101.inlet.flow_mol_phase_comp[0, "Vap", "toluene"]
        - m.fs.R101.outlet.flow_mol_phase_comp[0, "Vap", "toluene"]
    )
)

m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Vap", "benzene"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Vap", "toluene"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Vap", "hydrogen"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Vap", "methane"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Liq", "benzene"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Liq", "toluene"].fix(0.30)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Liq", "hydrogen"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Liq", "methane"].fix(1e-5)
m.fs.M101.toluene_feed.temperature.fix(303.2)
m.fs.M101.toluene_feed.pressure.fix(350000)

m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Vap", "benzene"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Vap", "toluene"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Vap", "hydrogen"].fix(0.30)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Vap", "methane"].fix(0.02)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Liq", "benzene"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Liq", "toluene"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Liq", "hydrogen"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Liq", "methane"].fix(1e-5)
m.fs.M101.hydrogen_feed.temperature.fix(303.2)
m.fs.M101.hydrogen_feed.pressure.fix(350000)

# Fix the temperature of the outlet from the heater H101
m.fs.H101.outlet.temperature.fix(600)

# Todo: Fix the 'conversion' of the reactor R101
m.fs.R101.conversion.fix(0.75)

# Todo: Fix the 'heat_duty' of the reactor R101
m.fs.R101.heat_duty.fix(0)

# Fix the temperature of the vapor outlet from F101
m.fs.F101.vap_outlet.temperature.fix(325.0)

# Fix the pressure drop in the flash F101
m.fs.F101.deltaP.fix(0)

# Fix the split fraction of the 'purge' stream from S101
m.fs.S101.split_fraction[0, "purge"].fix(0.2)

# Fix the pressure of the outlet from the compressor C101
m.fs.C101.outlet.pressure.fix(350000)

# Fix the temperature of the outlet from the heater H102
m.fs.H102.outlet.temperature.fix(375)

# Fix the pressure drop in the heater H102
m.fs.H102.deltaP.fix(-200000)

'fs.BTHM_params.pressure_crit' to mutable.
'fs.BTHM_params.temperature_crit' to mutable.
'fs.BTHM_params.mw_comp' to mutable.
'fs.BTHM_params.dens_liq_param_1' to mutable.
'fs.BTHM_params.dens_liq_param_2' to mutable.
'fs.BTHM_params.dens_liq_param_3' to mutable.
'fs.BTHM_params.dens_liq_param_4' to mutable.
'fs.BTHM_params.temperature_boil' to mutable.
'fs.BTHM_params.cp_ig_1' to mutable.
'fs.BTHM_params.cp_ig_2' to mutable.
'fs.BTHM_params.cp_ig_3' to mutable.
'fs.BTHM_params.cp_ig_4' to mutable.
'fs.BTHM_params.cp_ig_5' to mutable.
'fs.BTHM_params.pressure_sat_coeff_A' to mutable.
'fs.BTHM_params.pressure_sat_coeff_B' to mutable.
'fs.BTHM_params.pressure_sat_coeff_C' to mutable.
'fs.BTHM_params.dh_vap' to mutable.
'fs.BT_params.pressure_critical' to mutable.
'fs.BT_params.temperature_critical' to mutable.
'fs.BT_params.mw_comp' to mutable.
'fs.BT_params.dh_form' to mutable.
'fs.BT_params.ds_form' to mutable.


In [20]:
# Set scaling factors for heat duty, reaction extent and volume
iscale.set_scaling_factor(m.fs.H101.control_volume.heat, 1e-2)
iscale.set_scaling_factor(m.fs.R101.control_volume.heat, 1e-2)
iscale.set_scaling_factor(m.fs.R101.control_volume.rate_reaction_extent, 1)
iscale.set_scaling_factor(m.fs.R101.control_volume.volume, 1)
iscale.set_scaling_factor(m.fs.F101.control_volume.heat, 1e-2)
iscale.set_scaling_factor(m.fs.H102.control_volume.heat, 1e-2)

# iscale.set_scaling_factor(m.fs.F101.control_volume.properties_out[0.0].pressure_sat['hydrogen'], 8.234E+07)
# iscale.set_scaling_factor(m.fs.H101.control_volume.properties_out[0.0].pressure_sat['hydrogen'], 8.592E+07)
iscale.set_scaling_factor(m.fs.R101.control_volume.properties_out[0.0].temperature, 5.811E-05)
iscale.set_scaling_factor(m.fs.F101.control_volume.properties_in[0.0].temperature, 5.811E-05)
iscale.set_scaling_factor(m.fs.C101.ratioP[0.0],3.500E-05)
iscale.set_scaling_factor(m.fs.F101.control_volume.properties_out[0.0].pressure_sat['methane'], 3.343E06)
    
# iscale.calculate_scaling_factors(m.fs.H101)
# iscale.calculate_scaling_factors(m.fs.R101)
# iscale.calculate_scaling_factors(m.fs.F101)
# iscale.calculate_scaling_factors(m.fs.H102)
# iscale.calculate_scaling_factors(m.fs.S101)

In [21]:
seq = SequentialDecomposition()
seq.options.select_tear_method = "heuristic"
seq.options.tear_method = "Wegstein"
seq.options.iterLim = 3

# Using the SD tool
G = seq.create_graph(m)
heuristic_tear_set = seq.tear_set_arcs(G, method="heuristic")
order = seq.calculation_order(G)

tear_guesses = {
    "flow_mol_phase_comp": {
        (0, "Vap", "benzene"): 1e-5,
        (0, "Vap", "toluene"): 1e-5,
        (0, "Vap", "hydrogen"): 0.30,
        (0, "Vap", "methane"): 0.02,
        (0, "Liq", "benzene"): 1e-5,
        (0, "Liq", "toluene"): 0.30,
        (0, "Liq", "hydrogen"): 1e-5,
        (0, "Liq", "methane"): 1e-5,
    },
    "temperature": {0: 303},
    "pressure": {0: 350000},
}

# Pass the tear_guess to the SD tool
seq.set_guesses_for(m.fs.H101.inlet, tear_guesses)

from idaes.core.initialization.block_triangularization import BlockTriangularizationInitializer
initializer = BlockTriangularizationInitializer(constraint_tolerance=1e-4)
def function(unit):
    print(unit.name)
    if isinstance(unit, Heater):  # Heaters have their own initialization method
        unit.initialize(outlvl=idaeslog.INFO)
    else:
        status = initializer.initialize(unit, output_level=idaeslog.INFO)

In [60]:
seq = SequentialDecomposition()
seq.options.select_tear_method = "heuristic"
seq.options.tear_method = "Wegstein"
seq.options.iterLim = 3

# Using the SD tool
G = seq.create_graph(m)
heuristic_tear_set = seq.tear_set_arcs(G, method="heuristic")
order = seq.calculation_order(G)

tear_guesses = {
    "flow_mol_phase_comp": {
        (0, "Vap", "benzene"): 1e-5,
        (0, "Vap", "toluene"): 1e-5,
        (0, "Vap", "hydrogen"): 0.30,
        (0, "Vap", "methane"): 0.02,
        (0, "Liq", "benzene"): 1e-5,
        (0, "Liq", "toluene"): 0.30,
        (0, "Liq", "hydrogen"): 1e-5,
        (0, "Liq", "methane"): 1e-5,
    },
    "temperature": {0: 303},
    "pressure": {0: 350000},
}

# Pass the tear_guess to the SD tool
seq.set_guesses_for(m.fs.H101.inlet, tear_guesses)

def function(unit):
    print(unit.name)
    try:
        print(unit.default_initializer())
        initializer = unit.default_initializer()
        initializer.initialize(unit, output_level=idaeslog.INFO)
    except:
        print(f'{unit} did not initialize well')
        print(initializer.initialize(unit, output_level=idaeslog.INFO))
    # if isinstance(unit, Heater):  # Heaters have their own initialization method
    #     unit.initialize(outlvl=idaeslog.INFO)
    # else:
    #     status = initializer.initialize(unit, output_level=idaeslog.INFO)

In [61]:
seq.run(m, function)

fs.H101
fs.H101 did not initialize well


NotImplementedError: 

In [23]:
from idaes.core.util import DiagnosticsToolbox
dt=DiagnosticsToolbox(m)
dt.report_numerical_issues()

component keys that are not exported as part of the NL file.  Skipping.
Model Statistics

    Jacobian Condition Number: 2.551E+23

------------------------------------------------------------------------------------
4 WARNINGS


------------------------------------------------------------------------------------
6 Cautions

    Caution: 68 Variables with value close to their bounds (abs=1.0E-04, rel=1.0E-04)
    Caution: 67 Variables with value close to zero (tol=1.0E-08)
    Caution: 138 Variables with extreme value (<1.0E-04 or >1.0E+04)
    Caution: 99 Variables with extreme Jacobian values (<1.0E-04 or >1.0E+04)
    Caution: 63 Constraints with extreme Jacobian values (<1.0E-04 or >1.0E+04)
    Caution: 232 extreme Jacobian Entries (<1.0E-04 or >1.0E+04)

------------------------------------------------------------------------------------
Suggested next steps:

    display_constraints_with_large_residuals()
    display_variables_at_or_outside_bounds()
    display_variables_with_ex

In [24]:
dt.display_variables_with_extreme_jacobians()

component keys that are not exported as part of the NL file.  Skipping.
The following variable(s) are associated with extreme Jacobian values (<1.0E-04 or>1.0E+04):

    fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp[Liq,benzene]: 1.477E+11
    fs.R101.control_volume.properties_out[0.0].flow_mol_phase_comp[Liq,methane]: 1.267E+11
    fs.R101.control_volume.properties_out[0.0].flow_mol_phase_comp[Liq,hydrogen]: 1.267E+11
    fs.F101.control_volume.properties_in[0.0].flow_mol_phase_comp[Liq,methane]: 1.267E+11
    fs.F101.control_volume.properties_in[0.0].flow_mol_phase_comp[Liq,hydrogen]: 1.267E+11
    fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp[Liq,methane]: 8.886E+10
    fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp[Liq,hydrogen]: 8.886E+10
    fs.C101.control_volume.properties_in[0.0].flow_mol_phase_comp[Liq,toluene]: 7.163E+10
    fs.M101.vapor_recycle_state[0.0].flow_mol_phase_comp[Liq,toluene]: 7.162E+10
    fs.C101.control_volum

In [7]:
m.fs.H101.control_volume.properties_out[0.0].equilibrium_constraint['benzene'].pprint()

{Member of equilibrium_constraint} : Size=4, Index=fs.BTHM_params.component_list, Active=True
    Key     : Lower : Body                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [38]:
(m.fs.H101.control_volume.properties_out[0.0].pressure_sat['benzene'].value*(m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Liq','benzene'].value/(m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Liq','benzene'].value + m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Liq','toluene'].value + m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Liq','methane'].value + m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Liq','hydrogen'].value))) - ((m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Vap','benzene'].value/(m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Vap','benzene'].value + m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Vap','toluene'].value + m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Vap','methane'].value + m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Vap','hydrogen'].value))*m.fs.H101.control_volume.properties_out[0.0].pressure.value)

-0.0011934341309824958

In [44]:
print(m.fs.H101.control_volume.properties_in[0.0].flow_mol_phase_comp['Liq','hydrogen'].value)
print(m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Liq','hydrogen'].value)
print(m.fs.H101.control_volume.properties_in[0.0].flow_mol_phase_comp['Liq','methane'].value)
print(m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Liq','methane'].value)
print(m.fs.H101.control_volume.properties_in[0.0].flow_mol_phase_comp['Liq','benzene'].value)
print(m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Liq','benzene'].value)
print(m.fs.H101.control_volume.properties_in[0.0].flow_mol_phase_comp['Liq','toluene'].value)
print(m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Liq','toluene'].value)

2.0007997438714495e-05
1e-12
2.0007997438714495e-05
1e-12
2.0007997438714495e-05
1.3034935871152319e-07
0.30001000799734406
8.414878009343252e-07


In [49]:
print(m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Vap','hydrogen'].value)

0.562139567854012


In [47]:
m.fs.H101.control_volume.properties_in[0.0].temperature.pprint()

temperature : State temperature
    Size=1, Index=None, Units=K
    Key  : Lower : Value              : Upper : Fixed : Stale : Domain
    None :   298 : 314.07480065353843 :  1000 : False :  True : NonNegativeReals


In [ ]:
(m.fs.H101.control_volume.properties_out[0.0].pressure_sat['benzene'].value*(m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Liq','benzene'].value/(m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Liq','benzene'].value + m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Liq','toluene'].value + m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Liq','methane'].value + m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Liq','hydrogen'].value))) - ((m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Vap','benzene'].value/(m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Vap','benzene'].value + m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Vap','toluene'].value + m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Vap','methane'].value + m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Vap','hydrogen'].value))*m.fs.H101.control_volume.properties_out[0.0].pressure.value)

In [36]:
1000.0*m.fs.H101.control_volume.properties_out[0.0].enth_mol_phase_comp['Liq','hydrogen'].value - (m.fs.BTHM_params.cp_ig_5['Liq','hydrogen'].value/5*(m.fs.H101.control_volume.properties_out[0.0].temperature.value**5 - m.fs.BTHM_params.temperature_ref.value**5) + m.fs.BTHM_params.cp_ig_4['Liq','hydrogen'].value/4*(m.fs.H101.control_volume.properties_out[0.0].temperature.value**4 - m.fs.BTHM_params.temperature_ref.value**4) + m.fs.BTHM_params.cp_ig_3['Liq','hydrogen'].value/3*(m.fs.H101.control_volume.properties_out[0.0].temperature.value**3 - m.fs.BTHM_params.temperature_ref.value**3) + m.fs.BTHM_params.cp_ig_2['Liq','hydrogen'].value/2*(m.fs.H101.control_volume.properties_out[0.0].temperature.value**2 - m.fs.BTHM_params.temperature_ref.value**2) + m.fs.BTHM_params.cp_ig_1['Liq','hydrogen'].value*(m.fs.H101.control_volume.properties_out[0.0].temperature.value - m.fs.BTHM_params.temperature_ref.value))

0.0

In [8]:
m.fs.s03_expanded.pprint()

s03_expanded : Size=1, Index=None, Active=True
    3 Constraint Declarations
        flow_mol_phase_comp_equality : Size=8, Index=fs._time*fs.BTHM_params.phase_list*fs.BTHM_params.component_list, Active=True
            Key                      : Lower : Body                                                                                                                                     : Upper : Active
             (0.0, 'Liq', 'benzene') :   0.0 :   fs.M101.mixed_state[0.0].flow_mol_phase_comp[Liq,benzene] - fs.H101.control_volume.properties_in[0.0].flow_mol_phase_comp[Liq,benzene] :   0.0 :   True
            (0.0, 'Liq', 'hydrogen') :   0.0 : fs.M101.mixed_state[0.0].flow_mol_phase_comp[Liq,hydrogen] - fs.H101.control_volume.properties_in[0.0].flow_mol_phase_comp[Liq,hydrogen] :   0.0 :   True
             (0.0, 'Liq', 'methane') :   0.0 :   fs.M101.mixed_state[0.0].flow_mol_phase_comp[Liq,methane] - fs.H101.control_volume.properties_in[0.0].flow_mol_phase_comp[Liq,methane] :   0

In [9]:
m.fs.M101.mixed_state[0.0].temperature.pprint()

temperature : State temperature
    Size=1, Index=None, Units=K
    Key  : Lower : Value             : Upper : Fixed : Stale : Domain
    None :   298 : 314.0955643806385 :  1000 : False :  True : NonNegativeReals


In [10]:
m.fs.H101.control_volume.properties_in[0.0].temperature.pprint()

temperature : State temperature
    Size=1, Index=None, Units=K
    Key  : Lower : Value             : Upper : Fixed : Stale : Domain
    None :   298 : 314.0747969291496 :  1000 : False :  True : NonNegativeReals


In [11]:
m.fs.H101.control_volume.properties_out[0.0].temperature.pprint()

temperature : State temperature
    Size=1, Index=None, Units=K
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :   298 :   600 :  1000 :  True :  True : NonNegativeReals


In [12]:
m.fs.H101.control_volume.pprint()

control_volume : Size=1, Index=None, Active=True
    2 Var Declarations
        heat : Heat transferred into control volume
            Size=1, Index=fs._time, Units=kg*m**2/s**3
            Key : Lower : Value             : Upper : Fixed : Stale : Domain
            0.0 :  None : 47334.81195332139 :  None : False :  True :  Reals
        phase_equilibrium_generation : Amount of generation in control volume by phase equilibria
            Size=4, Index=fs._time*fs.BTHM_params.phase_equilibrium_idx, Units=mol/s
            Key      : Lower : Value                  : Upper : Fixed : Stale : Domain
            (0.0, 1) :  None : 1.9877650660161695e-05 :  None : False :  True :  Reals
            (0.0, 2) :  None :     0.3000091665124016 :  None : False :  True :  Reals
            (0.0, 3) :  None : 2.0007999028186373e-05 :  None : False :  True :  Reals
            (0.0, 4) :  None : 2.0007999028186373e-05 :  None : False :  True :  Reals

    3 Constraint Declarations
        enthalpy_b

In [13]:
(((m.fs.H101.control_volume.properties_in[0.0].flow_mol_phase_comp['Liq','benzene'].value + m.fs.H101.control_volume.properties_in[0.0].flow_mol_phase_comp['Liq','toluene'].value + m.fs.H101.control_volume.properties_in[0.0].flow_mol_phase_comp['Liq','methane'].value + m.fs.H101.control_volume.properties_in[0.0].flow_mol_phase_comp['Liq','hydrogen'].value)*m.fs.H101.control_volume.properties_in[0.0].enth_mol_phase['Liq'].value) + ((m.fs.H101.control_volume.properties_in[0.0].flow_mol_phase_comp['Vap','benzene'].value + m.fs.H101.control_volume.properties_in[0.0].flow_mol_phase_comp['Vap','toluene'].value + m.fs.H101.control_volume.properties_in[0.0].flow_mol_phase_comp['Vap','methane'].value + m.fs.H101.control_volume.properties_in[0.0].flow_mol_phase_comp['Vap','hydrogen'].value)*m.fs.H101.control_volume.properties_in[0.0].enth_mol_phase['Vap'].value) - (((m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Liq','benzene'].value + m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Liq','toluene'].value + m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Liq','methane'].value + m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Liq','hydrogen'].value)*m.fs.H101.control_volume.properties_out[0.0].enth_mol_phase['Liq'].value) + ((m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Vap','benzene'].value + m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Vap','toluene'].value + m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Vap','methane'].value + m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Vap','hydrogen'].value)*m.fs.H101.control_volume.properties_out[0.0].enth_mol_phase['Vap'].value)) + m.fs.H101.control_volume.heat[0.0].value)

-3.6307028494775295e-09

In [14]:
(m.fs.H101.control_volume.properties_in[0.0].flow_mol_phase_comp['Liq','hydrogen'].value - m.fs.H101.control_volume.properties_out[0.0].flow_mol_phase_comp['Liq','hydrogen'].value + (0*m.fs.H101.control_volume.phase_equilibrium_generation[0.0,1].value + 0*m.fs.H101.control_volume.phase_equilibrium_generation[0.0,2].value - m.fs.H101.control_volume.phase_equilibrium_generation[0.0,3].value + 0*m.fs.H101.control_volume.phase_equilibrium_generation[0.0,4].value))

-2.8161118050115327e-14